In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!nvidia-smi

Fri Jun 11 14:28:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import keras
from keras.models import Sequential
from keras.layers import  Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [13]:
# Initialising the CNN
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=(300, 300, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss=keras.metrics.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 296, 296, 32)      2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 144, 144, 32)      25632     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 165888)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               19906680  
_________________________________________________________________
dense_7 (Dense)              (None, 84)               

In [14]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/CNN/dataset/training_set',
                                                 target_size = (300, 300),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/CNN/dataset/test_set',
                                            target_size = (300, 300),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8048 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [15]:
classifier = model.fit(training_set,
                         steps_per_epoch = len(training_set),
                         epochs = 250,
                         validation_data = test_set,    
                         validation_steps = len(test_set))

Epoch 1/250
252/252 [==============================] - 149s 590ms/step - loss: 0.9333 - accuracy: 0.4861 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 2/250
252/252 [==============================] - 148s 589ms/step - loss: 0.6933 - accuracy: 0.4830 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/250
252/252 [==============================] - 148s 587ms/step - loss: 0.6932 - accuracy: 0.5041 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/250
252/252 [==============================] - 148s 587ms/step - loss: 0.6934 - accuracy: 0.5075 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/250
252/252 [==============================] - 148s 588ms/step - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6878 - val_accuracy: 0.5595
Epoch 6/250
252/252 [==============================] - 148s 589ms/step - loss: 0.6924 - accuracy: 0.5181 - val_loss: 0.6929 - val_accuracy: 0.5495
Epoch 7/250
252/252 [==============================] - 148s 589ms/step - loss: 0.6790 - accuracy: 0.5724 - val_loss: 0

In [16]:
model.save("LeNet.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
# load model
model_load = load_model('LeNet.h5')

In [ ]:
test_image = image.load_img('/content/drive/MyDrive/CNN/dataset/single_prediction/cat_or_dog_11.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model_load.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

dog
